## Sandbox for marching squares

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from marching_squares import Grid, UnitCell, Lattice
from tables import STATES

In [2]:
def center_ellipse(x,y,r,c):

    draw.ellipse([x - r, y - r, x + r, y + r],fill=c)

def center_rectangle(x,y,l,w,c):
    l = l/2
    w = w/2
    draw.rectangle([x - w, y - l, x + w, y + l],fill=c)

Marching squares

In [16]:
def get_state(a,b,c,d):
    return a * 8 + b * 4 + c * 2 + d * 1

def field_1(x, y, radius):
    # distance from origin
    d = np.sqrt(x ** 2 + y ** 2)
    if d < radius:
        return 1
    else:
        return 0
def field_2(x, y, radius):
    # distance from center point
    d = np.sqrt((x - 1080 / 2) ** 2 + (y - 1080 / 2) ** 2)
    if d < radius:
        return 1
    else:
        return 0

In [24]:
image_resolution = 1080

img = Image.new('RGB', (image_resolution, image_resolution))
draw = ImageDraw.Draw(img)

grid_divisions = 20
grid_scale = image_resolution / (grid_divisions - 1)


grid = Grid(grid_scale, grid_divisions, grid_divisions)

for point in grid.points:
    v = field_2(point[0], point[1], 800)
    if v == 0:
        color = "white"
    else:
        color = "red"

    scale = grid.scale
    x = point[0]
    y = point[1]

    ## values at points
    v0 = field_2(x,y, radius)
    v1 = field_2(x + scale, y, radius)
    v2 = field_2(x + scale, y + scale, radius)
    v3 = field_2(x, y + scale, radius)

    ## edge point locations
    a = [x + scale / 2, y]
    b = [x + scale, y + scale / 2]
    c = [x + scale / 2, y + scale]
    d = [x, y + scale / 2]

    edge_points = [a,b,c,d]

    radius = 400
    state = get_state(v0, v1, v2, v3)

    edges = STATES[state]

    for line in edges:
        p1 = edge_points[line[0]]
        p2 = edge_points[line[1]]
        draw.line([p1[0], p1[1], p2[0], p2[1]], fill="white",width=10)

    center_ellipse(point[0], point[1], 8, color)


img.save('pixels.png')

Lattices

In [5]:
# image_resolution = 1080

# img = Image.new('RGB', (image_resolution, image_resolution))
# draw = ImageDraw.Draw(img)

# def draw_cell(unit_cell, coords):
#     for beam in unit_cell.beams:
#         x1 = coords[beam[0]][0]
#         y1 = coords[beam[0]][1]
#         x2 = coords[beam[1]][0]
#         y2 = coords[beam[1]][1]
#         draw.line([x1, y1, x2, y2], fill="white",width=10)

# unit_cell = UnitCell("SC")
# scale = 100
# unit_cell.x_scale = scale
# unit_cell.y_scale = scale
# lattice = Lattice(unit_cell, 5, 5)
# i = 0
# coords = lattice.return_cell_coords(0,0)
# for coords in lattice.return_all_cell_coords():
#     draw_cell(unit_cell, coords)
# # for coords in lattic coords)
# img.save('pixels.png')